In [1]:
import sys

import pandas as pd
import mne
import h5py

from os import listdir
from os.path import isfile, join
import re
import numpy as np
from pathlib import Path
import pandas as pd
import os
pfad = '/Users/Ajit/Documents/_Privat/TU Master/Module/Introduction to Physiological Computing/Neuro-Usability Project/emotional_classifier-master'
sys.path.append(pfad)
from src.utils import ensure_dir

## import time stamps and create array indices
creates a stamps list with dict objects
```
{'game_name': [[start_idx, end_idx]])}
games_names = ['beat', 'brook', 'nat']```

In [3]:
import re
from datetime import datetime
from datetime import timedelta
data_folder = pfad + "/data/"
timestamps = open(data_folder+"timestamps.txt", "r")
text = ''.join(timestamps.readlines())
str_stamps = re.compile('#+  \d+  #+').split(text)
sfreq = 512
stamps = []
for i, s in enumerate(str_stamps):
    beat, brook, nat = re.findall('.*beat', s), re.findall('.*brook', s), re.findall('.*nat', s)
    beat = [b.split('   beat')[0].split(' - ') for b in beat]
    brook = [b.split('   brook')[0].split(' - ') for b in brook]
    nat = [b.split('   nat')[0].split(' - ') for b in nat]
    beat = [[(datetime.strptime(t, '%M:%S') - datetime.strptime('00:00', '%M:%S')).total_seconds() for t in b] for b in beat]
    brook = [[(datetime.strptime(t, '%M:%S') - datetime.strptime('00:00', '%M:%S')).total_seconds() for t in b] for b in brook]
    nat = [[(datetime.strptime(t, '%M:%S') - datetime.strptime('00:00', '%M:%S')).total_seconds() for t in b] for b in nat]
    beat_trials_idx,brook_trials_idx, nat_trials_idx = [], [], []
    beat_trials_idx = [[int(b[0] * sfreq), int(b[1] * sfreq) ] for b in beat]
    brook_trials_idx = [[int(b[0] * sfreq), int(b[1] * sfreq)] for b in brook]
    nat_trials_idx = [[int(b[0] * sfreq), int(b[1] * sfreq)] for b in nat]
    trials_dict = {}
    if beat_trials_idx:
        trials_dict.update({'beat': beat_trials_idx})
    if brook_trials_idx:
        trials_dict.update({'brook': brook_trials_idx})
    if nat_trials_idx:
        trials_dict.update({'nat': nat_trials_idx})
    stamps.append(trials_dict)
stats = pd.read_csv(data_folder+"stats.csv", sep=';')



In [4]:
stamps

[{},
 {'beat': [[168960, 235520]],
  'brook': [[423936, 426496]],
  'nat': [[522240, 675840]]},
 {'beat': [[189440, 261120]],
  'brook': [[721920, 757760], [808960, 844800]],
  'nat': [[424960, 563200]]},
 {'beat': [[232960, 335360]],
  'brook': [[465920, 504320], [524800, 588800]],
  'nat': [[632320, 796160]]},
 {'beat': [[202240, 220160], [256000, 332800]],
  'brook': [[607232, 643072], [668672, 702464]],
  'nat': [[870400, 1024000]]},
 {'beat': [[560640, 637440]],
  'brook': [[775680, 806400]],
  'nat': [[929280, 1082880]]},
 {'beat': [[107520, 179200]],
  'brook': [[271360, 337920]],
  'nat': [[417280, 570880]]},
 {'beat': [[154624, 225792]],
  'brook': [[353280, 441344]],
  'nat': [[545280, 698880]]},
 {'beat': [[243200, 313344]],
  'brook': [[415744, 456704], [472064, 510464]],
  'nat': [[605184, 733184]]},
 {'beat': [[156672, 225280], [252416, 327680]],
  'brook': [[530944, 579584]],
  'nat': [[757248, 903168], [922624, 985088]]},
 {'beat': [[181248, 250880], [317440, 370176]],


### transform HDF5 files to csvs

In [5]:
def hdf5_to_csv(input_directory, output_directory):
    files = [f for f in listdir(input_directory) if
             isfile(join(input_directory, f)) and f.endswith(".hdf5")]

    for f in files:
        try:
            file = h5py.File(join(input_directory, f), 'r')
            trial_data = file['RawData']['Samples'][:].T
            f_name = "{}.csv".format(f.split('2019')[0].split('RecordSession_')[1].split('_')[0])
            np.savetxt(join(output_directory, f_name), trial_data, delimiter=',')
        except Exception as e:
            print(e, f)
input_directory, output_directory = data_folder+"hdf5", data_folder+"csv/"
ensure_dir(output_directory)
hdf5_to_csv(input_directory, output_directory)

Unable to open file (File signature not found) RecordSession_4_lanthao2019.01.30_16.26.03.hdf5


## load csv  trial data

In [6]:
trials = [1,2,3,6,7,8,9,10,11,12,13,14,15,16]
csv_data_path = data_folder+"csv/{}.csv"
# trial_paths = [join(csv_data_path, f) for f in listdir(csv_data_path) if
#                             isfile(join(csv_data_path, f)) and f.endswith(".csv")]
trial_data = {}
for t in trials:
    if isfile(csv_data_path.format(t)):
        data = np.loadtxt(csv_data_path.format(t), delimiter=',')
        trial_data.update({t: data})

In [8]:
stats = stats.rename(columns={ stats.columns[0]: "trial" })
stats

,trial,age,gender,gaming,vr_experience,beat_valence,beat_arousal,brook_valence,brook_arousal,nat_valence,nat_arousal
0,1,35,1,1,1,6,5,1,9,3,2
1,2,27,1,1,2,8,7,8,8,6,5
2,3,29,2,1,1,6,6,5,8,5,3
3,4,20,1,2,3,6,7,5,9,8,5
4,5,22,2,1,1,8,6,6,8,9,1
5,6,21,3,1,2,9,8,2,9,5,3
6,7,26,2,5,4,7,7,5,7,7,2
7,8,25,1,1,2,9,9,3,5,6,4
8,9,23,2,3,1,9,7,7,9,5,2
9,10,22,1,1,1,8,7,7,9,5,3


####  Create MNE Objects and channel mapping with events with valece, arousal scores

In [10]:
labeled_data = {'beat':[], 'brook': [], 'nat': []}
channel_names = ['F3', 'FZ', 'F4', 'C3', 'CZ', 'C4', 'P3', 'PZ','P4']
info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types="eeg")
for t in trials:
    bt_lbl, br_lbl, nat_lbl = np.split(stats.loc[stats['trial']==t][['beat_valence','beat_arousal','brook_valence','brook_arousal','nat_valence','nat_arousal']].values[0],3)
    for bt_t in stamps[t]['beat']:
        st, end = bt_t
        mne.io.RawArray(trial_data[t][:, st:end], info)
        labeled_data['beat'].append([mne.io.RawArray(trial_data[t][:, st:end], info), bt_lbl])
    for br_t in stamps[t]['brook']:
        st, end = br_t
        labeled_data['brook'].append([mne.io.RawArray(trial_data[t][:, st:end], info), br_lbl])
    for nat_t in stamps[t]['nat']:
        st, end = nat_t
        labeled_data['nat'].append([mne.io.RawArray(trial_data[t][:, st:end], info), nat_lbl])

import pickle
pickle.dump(labeled_data, open(data_folder+"labeled_data.pkl", "wb"))


Creating RawArray with float64 data, n_channels=9, n_times=66560
    Range : 0 ... 66559 =      0.000 ...   129.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=66560
    Range : 0 ... 66559 =      0.000 ...   129.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=2560
    Range : 0 ... 2559 =      0.000 ...     4.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=153600
    Range : 0 ... 153599 =      0.000 ...   299.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=71680
    Range : 0 ... 71679 =      0.000 ...   139.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=71680
    Range : 0 ... 71679 =      0.000 ...   139.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=35840
    Range : 0 ... 35839 =      0.000 ...    69.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=35840
    Range : 0 ... 35839 =      0.000 ..

Ready.
Creating RawArray with float64 data, n_channels=9, n_times=79872
    Range : 0 ... 79871 =      0.000 ...   155.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=32256
    Range : 0 ... 32255 =      0.000 ...    62.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=32256
    Range : 0 ... 32255 =      0.000 ...    62.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=29696
    Range : 0 ... 29695 =      0.000 ...    57.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=19456
    Range : 0 ... 19455 =      0.000 ...    37.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=97280
    Range : 0 ... 97279 =      0.000 ...   189.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=83456
    Range : 0 ... 83455 =      0.000 ...   162.998 secs
Ready.
Creating RawArray with float64 data, n_channels=9, n_times=83456
    Range : 0 ... 83455 =      0